# Distribution of vegetation in a landscape of heterogeneous predation risk

In [16]:
# importing libraries

import numpy as np
from matplotlib import pyplot as plt
import math
from itertools import permutations as perm

# Notebook parameters

plt.rcParams['figure.figsize'] = [8, 8]
plt.rcParams['font.size'] = 20

## Defining initialisation functions

In [2]:
# Defining resource matrix

def veg(N):
    
    # Starting with resoource available everywhere
    
    V = np.full((N, N), 1)

    return V

# test plot

plt.matshow(veg(50))
plt.colorbar()

In [4]:
# Defining fish matrix

def fish(N):

    # Defining a N x N matrix

    F =  np.zeros((N, N))

    # Placing 20 fish at the center

    c =  int(N/2)
    
    F[c, c] = 20

    return F

# test plot

plt.matshow(fish(50))
plt.colorbar()

In [6]:
# Defining predation risk maatrix

def risk(N):

    # defining a N x N matrix

    R =  np.zeros((N, N))

    # Creating a gradient of predation risk

    c = N/2 # center of the array

    for index, value in np.ndenumerate(R):

        # Calculating distinace from the center
    
        dist =  math.sqrt((c - index[0])**2 + (c - index[1])**2)

        # Asymptotic increase of predation risk from center 

        value = (dist/(10 + dist))*100

        # updating value
    
        R[index] = value

    return R

# test plot

plt.matshow(risk(50))
plt.colorbar()

## Defining update functions

In [120]:
# spread and regeneration of vegetation

def veg_update(veg_mat):

    N = np.shape(veg_mat) # getting shape of vegetation matrix

    for index, value in np.ndenumerate(veg_mat):

        if value == 0:

            adj = [] # empty list of adjascent cells

            # getting list of adjascent cells

            for dx in [-1, 0, 1]:

                for dy  in [-1, 0, 1]:

                    x = index[0] + dx # horizontal

                    y = index[1] + dy # vertical

                    if x > N[0]-1 or y > N[1]-1: # enforcing boundaries

                        break

                    else:
                
                        adj.append((x, y))

            # Spread of vegetation

            ## Getting values from adjascent cells

            adj_veg = []

            for cell in adj:

                adj_veg.append(veg_mat[cell])
            
            ## checking if neighbours are present

            if all(x == 0 for x in adj_veg):

                break
            
            else:
                
                # updating value 

                veg_mat[index] = int(np.random.choice(a = [0,1], p =  [0.5, 0.5])) # p = 0.5

    return(veg_mat)

# Test

V = veg(50)

for dx in range(10):

    for dy in range(10):

        V[dx, dy] = 0

plt.matshow(V)
plt.colorbar()

V = veg_update(V)

plt.matshow(V)
plt.colorbar()
